In [1]:
from mip import *

In [2]:
def ler_arquivo(file: str):
    restricoes = []
    objetivo = []
    n_var = 0
    n_rest = 0

    with open(file, "r") as arquivo:
        primeira_linha = arquivo.readline()
        info_quant = [int(n) for n in primeira_linha.split()]

        n_var = info_quant[0]
        n_rest = info_quant[1]
        
        objetivo = [int(n) for n in arquivo.readline().split()]
        
        for linha in arquivo:
            restricoes.append([int(n) for n in linha.split()])
    
    return n_var, n_rest, restricoes, objetivo
    

In [3]:
n_var, n_rest, restricoes, objetivo = ler_arquivo("teste1.txt")

print(f'Coeficientes da função objetivo: {objetivo}')
print(f'Coeficiente de restrições: {restricoes}')
print(f'Número de variáveis: {n_var}')
print(f'Número de restricoes: {n_rest}')    

Coeficientes da função objetivo: [2, 10, 8, 7, 10, 10, 6]
Coeficiente de restrições: [[5, 7, 8, 1, 7, 5, 6, 20], [1, 6, 4, 9, 10, 6, 10, 30], [4, 4, 4, 1, 5, 5, 10, 40], [3, 10, 8, 1, 3, 3, 8, 30], [10, 8, 9, 9, 7, 6, 10, 20], [6, 6, 3, 6, 3, 7, 2, 80], [7, 10, 7, 8, 7, 8, 7, 100], [9, 8, 1, 1, 8, 10, 2, 90], [1, 5, 3, 10, 2, 4, 9, 70], [9, 6, 1, 4, 7, 5, 10, 60], [5, 7, 4, 4, 3, 4, 10, 40]]
Número de variáveis: 7
Número de restricoes: 11


In [4]:
m = Model(sense=MAXIMIZE, solver_name=CBC)

# Criando variáveis
x = [m.add_var(var_type=CONTINUOUS,name=f'x_{i+1}') for i in range(n_var)]

# Função objetivo:
m.objective = xsum(x[i] * objetivo[i] for i in range(n_var))

# Adicionando restricoes:
for r in range(n_rest):
    m += xsum(x[i] * restricoes[r][i] for i in range(n_var)) <= restricoes[r][-1]


In [5]:
def show(m):
    m.optimize()

    print(f"Solution value  = {m.objective_value:.4f}\n")

    print(f"type: {m.vars}")

    print("Solution:")
    for v in m.vars:
        print(f"{v.name} = {v.x:.4f}")

In [6]:
def allIntegers(vars):
    return all(n.x.is_integer() for n in vars)